In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Activation, Dense, Input, LSTM
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Embedding
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.models import Model
from keras.models import Sequential
from keras.datasets import mnist
from keras.models import load_model
from keras.losses import binary_crossentropy,mean_squared_error
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.utils.np_utils as np_utils
from sklearn.model_selection import train_test_split
from IPython.core.debugger import set_trace
from keras.preprocessing.text import Tokenizer
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import argparse
import pandas as pd


def split_dataset(filename, test_split_ratio):
    df = pd.read_csv(filename, header=None)
    X_train, X_test, y_train, y_test = train_test_split(df[1], df[0], test_size=test_split_ratio, random_state=42)
    return X_train, X_test, y_train, y_test

In [37]:
if __name__ == '__main__':
    df = pd.read_csv("clean_data.csv", header=None)
    X_train, X_test, y_train, y_test = train_test_split(df[1], df[0], test_size=0.10, random_state=42)


    print((X_train[0]))

    embedding = np.load('300_glove_davidson.npy')
    
    t = Tokenizer()
    t.fit_on_texts(X_train)
    vocab_size = len(t.word_index) + 1
    print(vocab_size)
    encoded_docs = t.texts_to_sequences(X_train)

    max_length = 4
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    '''
    t = Tokenizer()
    t.fit_on_texts(X_test)
    vocab_size_test = len(t.word_index) + 1'''
    encoded_docs_test = t.texts_to_sequences(X_test)

    padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
    
    model=Sequential()  

    # EMBEDDING LAYER - DISTRIBUTED REPRESENTATION OF TWEETS 
    # EMBEDDINGS - GLOVE 100 dimensions further trained on davidson and heot dataset after proper preprocessing
    # EMBEDDING DIMENSION = 100
    print(np.shape(embedding)[0])
    print(vocab_size)
    model.add(Embedding(22141, 300, weights=[embedding], input_length=4, name='embedding_layer'))

    # Dropout Layer to reduce overfitting 
    model.add(Dropout(0.4))

    # LSTM Layer (2 LSTM layers preferable) - Units : 64
    model.add(LSTM(64,dropout_W=0.2,dropout_U=0.2))

    #Series of dense layers  
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax',name='last'))

    # Compiling Model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    # ONE HOT ENCODING - for traing and testing data and transfer learning
    y_train1 = np_utils.to_categorical(y_train, num_classes=3)
    y_test1 = np_utils.to_categorical(y_test, num_classes=3)
    #Y_transfer_learning_train=np_utils.to_categorical(Y_transfer_learning_train,num_classes=3)
    #Y_transfer_learning_test=np_utils.to_categorical(Y_transfer_learning_test,num_classes=3)
    print(np.shape(padded_docs))
    # Training Model
    #model.fit(padded_docs,y_train,epochs=1,batch_size=128)
    print(np.shape(y_train1))
    history = model.fit(padded_docs, y_train1, batch_size=32,epochs=2,validation_split=0.0)
    print(np.shape(y_train1))
    #testing
    co=0
    tot=0
    ans = model.predict([padded_docs_test])
    print(np.shape(y_test1))
    for i in range(len(y_test1)):
        tot+=1
        #print("X=%s, Predicted=%s, 1=%s, 2=%s" % (y_test1[i], ans[i], np.argmax(y_test1[i]), np.argmax(ans[i])))
        if np.argmax(ans[i]) == np.argmax(y_test1[i]):
            co+=1
    print(co)
    print(tot)
    acc = co / tot
    print(acc)

 usermention as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out
20796
22141
20796


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, dropout=0.2, recurrent_dropout=0.2)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 4, 300)            6642300   
_________________________________________________________________
dropout_10 (Dropout)         (None, 4, 300)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dense_19 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_20 (Dense)             (None, 32)                2080      
_________________________________________________________________
last (Dense)                 (None, 3)                 99        
Total params: 6,742,079
Trainable params: 6,742,079
Non-trainable params: 0
_________________________________________________________________
